# Text Mining Final Project 2019 - 2020 - LSTM

## Identifying Authors by Their Writings 

## Authors: 
- Lara Neves (m20190867) 
- Susana Paço (m20190821)
- Inês Diogo (m20190301)


**Objective:** To identify the authors of portuguese texts by training a model with labeled texts from the same authors. 



**First Visual Analysis**
* Metadata was detected on the top of most txt's which can turn the bias up in the models. It was agreed that a good idea was to remove this crucial metadata from the corpora and test if it made a significant difference in the final result:
    + The name of the authors was detected in most of the metadata;
    + References to the authors works was also detected.
* Texts from different eras of portuguese 


In [1]:
#installs - please uncomment those packages that you don't have within your system in order to install them

import sys
#!{sys.executable} -m pip install -U unidecode
#!{sys.executable} -m pip install -U keras
#!{sys.executable} -m pip install -U tensorflow
#!{sys.executable} -m pip install -U nltk
#!{sys.executable} -m pip install git+https://github.com/textpipe/textpipe.git
#!{sys.executable} -m pip install -U spacy
#!{sys.executable} -m  spacy download pt_core_news_sm

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from collections import defaultdict
from pathlib import Path

import os
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
from sklearn.metrics import f1_score


* [1. Data Pre-Processing](#DPP)
    
    * [1.1. Renaming txt Files](#rename)

    * [1.2. Extracting Data](#extract)
    
    * [1.3. Clearing MetaData](#ClearMD)
    
* [2. Creating a Baseline](#Baseline)

<a class="anchor" id="DPP">

## 1. Data Preprocessing



<a class="anchor" id="rename">

### 1.1. Renaming .txt Files


In [3]:
#Changing the names of the .txt files so there's no duplicates and we create a standardized form to 
#identify each .txt

def change_file_name(author):
    i = 0
    my_dir_path = "Data/Corpora/train/" + author
    
    for filename in os.listdir(my_dir_path): 
        
        #Define the new and old names with directory path
        new_name =str(author) + str(i) + ".txt"
        old_name = my_dir_path + '/' + filename 
        new_name = my_dir_path + '/' + new_name 
        
        #So it doesn't give out an error when it runs for the second time
        # rename all the files 
        if new_name != old_name: #IT STILL GIVES OUT ERROR
            os.rename(old_name, new_name) 
        i += 1

In [4]:
authors = ['AlmadaNegreiros','CamiloCasteloBranco','EcaDeQueiros','JoseRodriguesSantos','JoseSaramago','LuisaMarquesSilva']
authors_sigla = ['AN','CCB','EQ','JRS','JS','LMS']



#### ONLY RUN ONCE IF THE FILE NAMES ARE THE ORIGINAL otherwise, running a second time, will give an error

In [5]:
#for a in range(len(authors)):
#    change_file_name(authors[a])

<a class="anchor" id="extract">

### 1.2. Extracting Data

In [6]:
#Create a df for one author the respective .txt files in the corpora
def create_df_from_txt(author):
    my_dir_path = "Data/Corpora/train/" + author

    results = defaultdict(list)
    for file in Path(my_dir_path).iterdir():
        with open(file, "r",encoding = 'utf8') as file_open:
            results["id"].append(file.name)
            results["text"].append(file_open.read())
            results["author"] = author
            file_open.close()
    df = pd.DataFrame(results)
    
    return df

In [7]:
#Join together the dataframes from all the authors
def join_df(authors):

    df = pd.DataFrame()
    for a in range(len(authors)):
        df = df.append(create_df_from_txt(authors[a]))
    df.reset_index(inplace = True, drop = True)    
    return df 

In [8]:
### Running all the functions

#Creating the training data frame
traindf = join_df(authors)

#Replacing the name of the authors with labels of their initials
for i in range(0,len(authors)):
    traindf.author = traindf.author.replace(authors[i],authors_sigla[i])
traindf

,id,text,author
0,AlmadaNegreiros3.txt,\nTitle: Litoral\n A Amadeo de Souza Car...,AN
1,AlmadaNegreiros2.txt,\n\nTitle: A Invenção do Dia Claro\n\nAuthor: ...,AN
2,AlmadaNegreiros0.txt,Title: A Scena do Odio\n\nAuthor: José de Alma...,AN
3,AlmadaNegreiros1.txt,Title: O Jardim da Pierrette\n\nAuthor: José d...,AN
4,AlmadaNegreiros5.txt,\n\n*JOSÉ DE ALMADA-NEGREIROS*\n\n\n*K4\n\no q...,AN
...,...,...,...
58,LuisaMarquesSilva3.txt,CONTROL Z\nChegou a hora de vos contar. Chegou...,LMS
59,LuisaMarquesSilva2.txt,O terrível caso do botão assassino\nLuísa Marq...,LMS
60,LuisaMarquesSilva0.txt,A BELA HISTÓRIA DE DINIS E BEATRIZ OU REQUIEM ...,LMS
61,LuisaMarquesSilva1.txt,\n\n\nAcabou-se!\nLuísa Marques da Silva\n\nTí...,LMS


# Test DF

In [9]:
#Create a df for one author the respective .txt files in the corpora
def create_df_from_txttest(numberofwords):
    my_dir_path = "Data/Corpora/test/"+ str(numberofwords)

    results = defaultdict(list)
    for file in Path(my_dir_path).iterdir():
        with open(file, "r",encoding = 'utf8') as file_open:
            results["id"].append(file.name)
            results["text"].append(file_open.read())
            results["numberofwords"] = numberofwords
            file_open.close()
    df = pd.DataFrame(results)
    
    return df

In [10]:
def join_dftest(numberofwords):

    df = pd.DataFrame()
    for a in range(len(numberofwords)):
        df = df.append(create_df_from_txttest(numberofwords[a]))
    df.reset_index(inplace = True, drop = True)    
    return df 


In [11]:
numberofwords = [1000, 500]


In [12]:
### Running all the functions

#Creating the training data frame
testdf = join_dftest(numberofwords)



In [13]:
testdf

,id,text,numberofwords
0,text6.txt,"""O Senhor ensina pela pena o que o homem não s...",1000
1,text5.txt,"O cahos de cima a descer, a descer com a morta...",1000
2,text4.txt,"Agora, porém, era sem fervor, arrastadamente, ...",1000
3,text1.txt,"Depois, pouco a pouco, a tranquilidade regress...",1000
4,text3.txt,"Quase um mês depois, a época de exames aproxim...",1000
5,text2.txt,Justamente como se eu tivesse tido a ideia de ...,1000
6,text6.txt,"""O Senhor ensina pela pena o que o homem não s...",500
7,text5.txt,"O cahos de cima a descer, a descer com a morta...",500
8,text4.txt,"Agora, porém, era sem fervor, arrastadamente, ...",500
9,text1.txt,"Depois, pouco a pouco, a tranquilidade regress...",500


From analysis of the texts, these are the supposed y_test values

In [30]:
# fill a y_test column with possible answers



testdf.loc[testdf['id'] == 'text1.txt', 'possibleanswer'] = 'JS'
testdf.loc[testdf['id'] == 'text2.txt', 'possibleanswer'] = 'AN'
testdf.loc[testdf['id'] == 'text3.txt', 'possibleanswer'] = 'unknown'
testdf.loc[testdf['id'] == 'text4.txt', 'possibleanswer'] = 'EQ'
testdf.loc[testdf['id'] == 'text5.txt', 'possibleanswer'] = 'CCB'
testdf.loc[testdf['id'] == 'text6.txt', 'possibleanswer'] = 'JRS'




        

In [31]:
testdf

,id,text,numberofwords,possibleanswer,removeMetadata
0,text6.txt,"""O Senhor ensina pela pena o que o homem não s...",1000,JRS,"""O Senhor ensina pela pena o que o homem não s..."
1,text5.txt,"O cahos de cima a descer, a descer com a morta...",1000,CCB,"O cahos de cima a descer, a descer com a morta..."
2,text4.txt,"Agora, porém, era sem fervor, arrastadamente, ...",1000,EQ,"Agora, porém, era sem fervor, arrastadamente, ..."
3,text1.txt,"Depois, pouco a pouco, a tranquilidade regress...",1000,JS,"Depois, pouco a pouco, a tranquilidade regress..."
4,text3.txt,"Quase um mês depois, a época de exames aproxim...",1000,unknown,"Quase um mês depois, a época de exames aproxim..."
5,text2.txt,Justamente como se eu tivesse tido a ideia de ...,1000,AN,Justamente como se eu tivesse tido a ideia de ...
6,text6.txt,"""O Senhor ensina pela pena o que o homem não s...",500,JRS,"""O Senhor ensina pela pena o que o homem não s..."
7,text5.txt,"O cahos de cima a descer, a descer com a morta...",500,CCB,"O cahos de cima a descer, a descer com a morta..."
8,text4.txt,"Agora, porém, era sem fervor, arrastadamente, ...",500,EQ,"Agora, porém, era sem fervor, arrastadamente, ..."
9,text1.txt,"Depois, pouco a pouco, a tranquilidade regress...",500,JS,"Depois, pouco a pouco, a tranquilidade regress..."


<a class="anchor" id="clearMD">

### 1.2. Clearing MetaData
</a>

The majority of the .txt files have metadata at the beginning. This is unnecessary and may introduce noise in our model, as such it may be a good idea to remove it. 

In [16]:
#author names to remove them from metadata
authors = ["José de Almada Negreiros", "José de ALMADA-NEGREIROS", "JOSÉ DE ALMADA-NEGREIROS", "Almada Negreiros", "Camilo Castelo Branco", "CAMILLO CASTELLO BRANCO", "Eça de Queirós", "Eca de Queiros", "José Rodrigues dos Santos","Jose Rodrigues dos Santos", "JOSÉ RODRIGUES DOS SANTOS", "José Saramago", "Jose Saramago", "JoSÉ SaRamago", "Luísa Marques Silva", "Luisa Marques Silva", "Luísa Marques da Silva"]  

    

In [17]:
#book names to stop words
authorsandbooks = pd.read_excel('Data/AuthorsAndBooks.xlsx')
authorsandbooks.head(3)

,Eça de Queirós,Camilo Castelo Branco,Almada Negreiros,Saramago,José Rodrigues dos Santos,Luísa Marques Silva
0,O Mistério da Estrada de Sintra,Anátema,O Moinho,Terra do Pecado,"Comunicação, Difusão Cultural, 1992; Prefácio",Acabou-se!
1,O Crime do Padre Amaro,Os Mistérios de Lisboa,Os Outros,Manual de Pintura e Caligrafia,Crónicas de Guerra I - Da Crimeia a Dachau,Sete Histórias por Acontecer
2,A Tragédia da Rua das Flores,A Filha do Arcediago,"23, 2º Andar",Levantado do Chão,Crónicas de Guerra II - De Saigão a Bagdade,e-Medo


In [18]:
#authors work to arrays

# Eça Queirós books
Eca = authorsandbooks['Eça de Queirós']
Eca = Eca.dropna()
Eca = pd.array(Eca)


#Camilo Castelo Branco books
camilo = authorsandbooks['Camilo Castelo Branco']
camilo = camilo.dropna()
camilo = pd.array(camilo)


# Almada Negreiros books
Almada = authorsandbooks['Almada Negreiros']
Almada = Almada.dropna()
Almada = pd.array(Almada)


# Saramago books
Saramago = authorsandbooks['Saramago']
Saramago = Saramago.dropna()
Saramago = pd.array(Saramago)


# José Rodrigues dos Santos books
JRodriguesSantos = authorsandbooks['José Rodrigues dos Santos']
JRodriguesSantos = JRodriguesSantos.dropna()
JRodriguesSantos = pd.array(JRodriguesSantos)


# Luísa Marques Silva books
luisaMarquesSilva = authorsandbooks['Luísa Marques Silva']
luisaMarquesSilva = luisaMarquesSilva.dropna()
luisaMarquesSilva = pd.array(luisaMarquesSilva)

In [19]:
import re
from tqdm import tqdm_notebook as tqdm #progressbar

def removemetadata(doc):
    processed_corpus = []
    for i in tqdm(range(len(doc))):
        text = doc['text'].iloc[i,]
        for w in authors:
            text = re.sub(w,"",text)
        for x in Eca:
            text = re.sub(x,"",text)
        for t in camilo:
            text = re.sub(t,"",text)
        for s in Almada:
            text = re.sub(s,"",text)
        for y in Saramago:
            text = re.sub(y,"",text)
        for n in JRodriguesSantos:
            text = re.sub(n,"",text)
        for m in luisaMarquesSilva:
            text = re.sub(m,"",text)
        processed_corpus.append(text)
    return processed_corpus

In [20]:
#create a column to test the results of removing crucial metadata text
traindf['removeMetadata'] = removemetadata(traindf)
testdf['removeMetadata'] = removemetadata(testdf)

In [21]:
traindf.head()

,id,text,author,removeMetadata
0,AlmadaNegreiros3.txt,\nTitle: Litoral\n A Amadeo de Souza Car...,AN,\nTitle: \n \n\nAuthor: \n\nContributor:...
1,AlmadaNegreiros2.txt,\n\nTitle: A Invenção do Dia Claro\n\nAuthor: ...,AN,\n\nTitle: \n\nAuthor: \n\nRelease Date: Septe...
2,AlmadaNegreiros0.txt,Title: A Scena do Odio\n\nAuthor: José de Alma...,AN,Title: \n\nAuthor: \n\nRelease Date: September...
3,AlmadaNegreiros1.txt,Title: O Jardim da Pierrette\n\nAuthor: José d...,AN,Title: \n\nAuthor: \n\nRelease Date: September...
4,AlmadaNegreiros5.txt,\n\n*JOSÉ DE ALMADA-NEGREIROS*\n\n\n*K4\n\no q...,AN,\n\n**\n\n\n*K4\n\no quadrado\n\nAZUL*\n\nACAB...


<a class="anchor" id="clearTexts">

### 1.4. Cleaning Texts
</a>


In [22]:
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from tqdm import tqdm_notebook as tqdm #progressbar
from unidecode import unidecode
from nltk.stem import RSLPStemmer
#nltk.download('rslp')

#spacy tools
import pt_core_news_sm
import spacy
spacy_nlp = spacy.load('pt_core_news_sm')



#### Auxiliary functions for preprocessing:

In [23]:
def lowercasing(text):       
    text = text.lower()     
    return text

def to_string(text):
    text = text.split()
    return text

def lemmatization(word):
    lem = WordNetLemmatizer()
    word =lem.lemmatize(word)
    return word

def punctuation(word):
    word = re.sub('[\“\”\ "\-\'`~!@#$%^&*()_|+=?;:,.<>\{\}\[\]\\\/]','', word)
    return word

def stopwords_nltk(word):
    stop_words = set(stopwords.words("portuguese")) 
    return word in stop_words

def stopwords_spacy(word):
    return spacy_nlp.vocab[word].is_stop

def accents(word):
    word = unidecode(word)
    return word

### Preprocessing 1
##### with punctuation and no lemmatization

In [24]:
def preprocessing_with_punc(doc, column):
    processed_corpus = []
    for i in tqdm(range(len(doc))):
        text = doc[column].iloc[i,]    
    
        text = lowercasing(text)
        
        text = to_string(text)

        textfinal = []
        for word in text:
            if stopwords_nltk(punctuation(word)) or stopwords_spacy(punctuation(word)):
                word = re.sub('[^\“\”\ "\-\'`~!@#$%^&*()_|+=?;:,.<>\{\}\[\]\\\/]','', word)
            else:
                word
            textfinal.append(word)

        text = " ".join(textfinal)
    
        processed_corpus.append(text)
         
    return processed_corpus

In [25]:

#this process takes some time, please have patience
traindf['clean_with_punc'] = preprocessing_with_punc(traindf, 'removeMetadata')
testdf['clean_with_punc'] = preprocessing_with_punc(testdf, 'removeMetadata')

KeyboardInterrupt: 

### Preprocessing 2 
##### no punctuation and no lemmatization 

In [ ]:
def preprocessing_no_punc(doc, column):
    processed_corpus = []
    for i in tqdm(range(len(doc))):
        text = doc[column].iloc[i,]    
    
        text = lowercasing(text)
        
        text = to_string(text)
        
        text = [punctuation(word) for word in text]

        text = [word for word in text if (not stopwords_nltk(word)) and (not stopwords_spacy(word))]
        
        text = [accents(word) for word in text] 

        text = " ".join(text)
    
        processed_corpus.append(text)
         
    return processed_corpus

In [ ]:
traindf['clean_no_punc'] = preprocessing_no_punc(traindf, 'removeMetadata')
testdf['clean_no_punc'] = preprocessing_no_punc(testdf, 'removeMetadata')

## Dividing the text into chunks of 500 words

In [ ]:
def split_doc(doc,column,n):
    newdf = pd.DataFrame()
    newdf = newdf.reindex(columns = ['id','chunks','author']) 
    for i in tqdm(range(len(doc))):
        text = doc[column].iloc[i,]

        text = text.split()

        chunks = [' '.join(text[j:j+n]) for j in range(0,len(text),n)]

        for c in chunks:
            data = []
            values = [doc['id'].iloc[i,], c, doc['author'].iloc[i,]]
            a_dictionary = dict(zip(newdf.columns.tolist(), values))
            data.append(a_dictionary)
            newdf = newdf.append(data)
            
    newdf.index =[j for j in range(len(newdf))]  
              
    return newdf

In [ ]:
traindf_chunks = split_doc(traindf, 'clean_no_punc' ,500)


In [ ]:
traindf_chunks.head(10)


# LSTM

In [ ]:
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.utils import np_utils
import itertools
import pandas as pd
import numpy as np
import copy

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
testdf.head(1)

In [ ]:
traindf.head(1)


In [ ]:
traindf_chunks.head(1)

In [ ]:


#Tokenize sentences
print("Tokenizing in progress...")
#1. Train set
text_list_train = list(traindf_chunks['chunks'])
#text_list_train_lower = [word.lower() for word in text_list_train]
tokenized_text_train = [word_tokenize(i) for i in text_list_train]

#2. Test set
text_list_test = list(testdf['clean_no_punc'])
#text_list_test_lower = [word.lower() for word in text_list_test]
tokenized_text_test = [word_tokenize(i) for i in text_list_test]

#Create vocabulary from train set only
list_of_all_words= list(itertools.chain.from_iterable(tokenized_text_test))
vocabulary =sorted(list(set(list_of_all_words)))

#Remove stopwords (I found out that it makes no difference but you can try on your own)
#vocabulary = [word for word in vocabulary if word not in stopwords.words('english')]

#--------------------------Pre-processing train and test sets----------------------------------
print('Pre-processing Train set...')
tokenized_numbers_train = copy.deepcopy(tokenized_text_train)

i=-1
for list in tokenized_numbers_train:
    i=i+1
    j=-1
    for number in list:
        j = j + 1
        if tokenized_numbers_train[i][j] in vocabulary:
            tokenized_numbers_train[i][j]= vocabulary.index(number)
        else:
            tokenized_numbers_train[i][j] = 0

tokens_train = pd.DataFrame(tokenized_numbers_train, dtype='int32')
tokens_train = tokens_train.fillna(0)
tokens_train = tokens_train.astype(int)

print('Pre-processing Test set...')
tokenized_numbers_test = copy.deepcopy(tokenized_text_test)

i=-1
for list in tokenized_numbers_test:
    i=i+1
    j=-1
    for number in list:
        j = j + 1
        if tokenized_numbers_test[i][j] in vocabulary:
            tokenized_numbers_test[i][j] = vocabulary.index(number)
        else:
            tokenized_numbers_test[i][j] = 0

tokens_test = pd.DataFrame(tokenized_numbers_test, dtype='int32')
tokens_test = tokens_test.fillna(0)
tokens_test = tokens_test.astype(int)

print('Making some more pre-processing to train and test sets...')

#Bring both sets to same shape (Choose how many words to use)
max_words_in_sentence=30

#Shorten or extend Train set to reach selected length
if tokens_train.shape[1]>max_words_in_sentence:
    tokens_train = tokens_train.drop(tokens_train.columns[[range(max_words_in_sentence,tokens_train.shape[1])]], axis=1)
else:
    for col in range(tokens_train.shape[1],max_words_in_sentence):
        tokens_train[col]=0

#Shorten or extend Test set to reach selected length
if tokens_test.shape[1] > max_words_in_sentence:
    tokens_test = tokens_test.drop(tokens_test.columns[[range(max_words_in_sentence, tokens_test.shape[1])]],
                                     axis=1)
else:
    for col in range(tokens_test.shape[1], max_words_in_sentence):
        tokens_test[col] = 0

#------------------------------End of Pre-processing----------------------------------------------------

#Define train and Test sets
train_x = np.array(tokens_train)
train_y = np.array(traindf_chunks['author'])

test_x = np.array(tokens_test)

encoder1 = LabelEncoder()
encoder1.fit(train_y)
encoded_train_Y = encoder1.transform(train_y)
dummy_train_y = np_utils.to_categorical(encoded_train_Y)
dummy_train_y.astype(int)

l=len(vocabulary)+1
inp=train_x.shape[1]

#Build an LSTM model
model = Sequential()
model.add(Embedding(l, 64,input_length=inp))
model.add(LSTM(32, dropout=0.4, recurrent_dropout=0.1))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Fit the model
model.fit(train_x, dummy_train_y, epochs=15, batch_size=16, verbose=2)


# Predict and write to file
results = model.predict(test_x)
results = pd.DataFrame(results, columns=['AN','CCB','EQ','JRS','JS','LMS'])
results['prediction'] = results[['AN','CCB','EQ','JRS','JS','LMS']].idxmax(axis=1)
results ['possibleresult'] = y_test
results.insert(0, "id", id)




In [ ]:
model.evaluate(train_x, dummy_train_y)


In [ ]:
results